## Tips & techniques 

Profiling and optimizing code is a very broad subject, full of rabbit holes. The more you get into it the more you find out how much more you can optmize and how much better your tests could be done. It's also important to understand when to stop.

Careful to not fall into the over optimizing trap!

If you intend to open source your code, clarity might be much more relevant than squeezing every drop of performance of your scripts.

In this notebook I share a few things I picked up learning about scientific software performance. These were written with the Python scientific development context in mind. If you're doing something different like profiling web environment, for example, they might be completely irrelevant and several things might be very different. Use seu bom senso to guess which tips are relevant for each context.

#### #1 Generators are faster than list comprehensions

In [ ]:
from sys import getsizeof

comp = [i for i in range(10000)]
gen = (i for i in range(10000))

#gives size for list comprehension
x = getsizeof(comp)
print("x = ", x)

#gives size for generator expression
y = getsizeof(gen)
print("y = ", y)

In [ ]:
#List Comprehension:
import timeit

print(timeit.timeit('''list_com = [i for i in range(100) if i % 2 == 0]''', number=1000000))

In [ ]:
#Generator Expression:
import timeit

print(timeit.timeit('''gen_exp = (i for i in range(100) if i % 2 == 0)''', number=1000000))

The generator yields one item at a time and generates item only when in demand. Whereas, in a list comprehension, Python reserves memory for the whole list.

#### #2 Between `dict`, `list` and `set`, pick `set` for speed

If you can't use `set`s the next best thing might be a `dict`, depending on your user case. The following table can be found on the [Fluent Python](https://www.fluentpython.com) website:

<br>

len of array | dict time | set time | list time
-------------|-----------|----------|----------
1.000|0.099ms|0.107ms|9.115ms|
10.000|0.109ms|0.119ms|78.219ms|
100.000|0.156ms|0.147ms|767.975ms|
1.000.000|0.372ms|0.264ms|8.020.312ms|
10.000.000|0.512ms|0.330ms|78.558.771ms|

#### #3 Use `tuple` as Immutable Lists

The Python interpreter and standard library make extensive use of tuples as immutable lists, and so should you. This brings two key benefits:

- Clarity: When you see a tuple in code, you know its length will never change.
- Performance: A tuple uses less memory than a list of the same length, and it allows Python to do some optimizations.

#### #4 Chained comparisons are good

When comparing three variables with each other, instead of doing `x < y and y < z`, you can use `x < y < z`.
This should prove easier to read (more natural) and faster to run.

#### #5 When possible, sort by the key

When doing a custom sort on a list, try not to sort using a comparison function. Instead, when possible, sort by the key. This is because the key function will be called only once per item, whereas the comparison function will be called several times per item during the process. 

#### #6 Sorting can be very costly, learn about the main algorithms

[Here](https://realpython.com/sorting-algorithms-python/)'s a recommended source to learn about Python's implementation on some common algorithms.
If you prefer a more in depth approach, check the chapter on sorting of Introduction to Algorithms (Cormen, et al.).

#### #7 Sampling

Even if you have a lot of data, there might not be much advantage from using all of it. By sampling intelligently you might be able to derive the same insight from a much more manageable subset.

#### #8 I/O is very costly

- For compression you’ll probably find that you drop gzip and bz2, and embrace newer systems like lz4, snappy, and Z-Standard that provide better performance and random access.
- For storage formats you may find that you want self-describing formats that are optimized for random access, metadata storage, and binary encoding like Parquet, ORC, Zarr, HDF5, and GeoTIFF.
- When working on the cloud you may find that some older formats like HDF5 may not work as well.
- You may want to partition or chunk your data in ways that align well to common queries. In Dask DataFrame this might mean choosing a column to sort by for fast selection and joins. For Dask Array this might mean choosing chunk sizes that are aligned with your access patterns and algorithms.

#### #9 Your OS is providing you with an unstable environment

On top of everything you need to think about you also have to take into consideration that every little pertubation in your OS is like the clap of the wings of a butterfly. There are some softwares that might help you to minimize these interferences.

- [Tuna](https://linux.web.cern.ch/mrg/2/Tuna_User_Guide/)
- [isolcpus](https://www.linuxtopia.org/online_books/linux_kernel/kernel_configuration/re46.html)
- Blog post [series](https://vstinner.github.io/journey-to-stable-benchmark-system.html) on stabilizing your system

## Techniques

### Memoization

This is an optimization technique that saves the results of previous invocations of functions.

In [ ]:
# Fluent Python's implementation of a time function
import time

def clock(func):
    def clocked(*args):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [ ]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(4))

In [ ]:
import functools 

@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(4))

All the arguments taken by the decorated function must be hashable, because the underlying lru_cache uses a dict to store the results

`@cache` won't be available to you if your Python < 3.8, but you can still use `@lru_cache`.

`@lru_cache` contains two arguments, `maxsize` which receives an integer containing the maximum number of entries to be stored.

## Saving memory using slots

`__slots__` is a different way of storage model for instance attributes. It's stored in a hidden array that uses less memory than a `dict`.

Here we're going to take a look into how to use it and some unexpected behaviors it has.

In [ ]:
class Pixel:
    __slots__ = ('x', 'y')

p = Pixel()
p.__dict__

In [ ]:
p.x = 10
p.y = 12
p.color = 'red'

You need to declare all the attributes that are going to be used when the class is created.

👥 How can we fix this code?

Good, all this behavior was within the expected. Now let's take a look on what's weird about `__slots__`.

In [ ]:
class SpecializedPixel(Pixel):
    pass

sp = SpecializedPixel()
sp.__dict__

In [ ]:
sp.x = 10
sp.__dict__

In [ ]:
sp.color = 'white'
sp.__dict__

👥 What are the implications of having a `__dict__` here?

According to Fluent Python's profiling a list containing a custom array implementation of 10M instances using `__dict__` and `__slots__` offer very different results:

```bash
$ time python3 mem_test.py vector2d_v3
Selected Vector2d type: vector2d_v3.Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:      6,983,680
  Final RAM usage:  1,666,535,424
real 0m11.990s
user 0m10.861s
sys 0m0.978s
```

```bash
$ time python3 mem_test.py vector2d_v3_slots
Selected Vector2d type: vector2d_v3_slots.Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:      6,995,968
  Final RAM usage:    577,839,104
real 0m8.381s
user 0m8.006s
sys 0m0.352s
```